In [13]:
import cv2
import numpy as np
import os

def load_yolo_model(cfg_path, weights_path, names_path):
    """
    Load YOLO using OpenCV DNN module.
    """
    net = cv2.dnn.readNet(weights_path, cfg_path)
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    
    # Load class names
    with open(names_path, 'r') as f:
        classes = [line.strip() for line in f.readlines()]
    
    return net, output_layers, classes

def detect_objects(image, net, output_layers, conf_threshold=0.5):
    """
    Perform object detection.
    """
    height, width = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    detections = net.forward(output_layers)

    boxes = []
    confidences = []
    class_ids = []

    for detection in detections:
        for obj in detection:
            scores = obj[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > conf_threshold:
                center_x = int(obj[0] * width)
                center_y = int(obj[1] * height)
                w = int(obj[2] * width)
                h = int(obj[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                # Ensure bounding box stays within image bounds
                x = max(0, x)
                y = max(0, y)
                w = min(width - x, w)
                h = min(height - y, h)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, 0.4)
    filtered_boxes = [(class_ids[i], boxes[i]) for i in indices.flatten()]

    return filtered_boxes

def save_bounding_boxes(image, detections, classes, target_class_id, output_dir, prefix):
    """
    Saves bounding boxes of detected objects as individual images.
    """
    os.makedirs(output_dir, exist_ok=True)
    count = 0

    for class_id, bbox in detections:
        if class_id == target_class_id:
            # Extract bounding box details
            x, y, w, h = bbox

            # Clamp coordinates to image dimensions
            x = max(0, x)
            y = max(0, y)
            w = min(w, image.shape[1] - x)
            h = min(h, image.shape[0] - y)

            # Skip invalid bounding boxes
            if w <= 0 or h <= 0:
                print(f"Invalid bounding box skipped: {bbox}")
                continue

            # Crop the detected object
            cropped_img = image[y:y+h, x:x+w]

            # Define output path and save the cropped image
            output_path = os.path.join(output_dir, f"{prefix}_object_{count}.jpg")
            cv2.imwrite(output_path, cropped_img)
            print(f"Saved: {output_path}")  # Log saved image
            count += 1

def process_and_save(image_path, yolo_model, target_class_name, output_dir, prefix):
    """
    Detects objects in the image, crops and saves bounding boxes based on detected objects.
    """
    net, output_layers, classes = yolo_model

    # Load image
    image = cv2.imread(image_path)

    if image is None:
        print(f"Error loading image: {image_path}")
        return

    # Verify target class
    if target_class_name not in classes:
        print(f"Error: Class '{target_class_name}' not found in the provided classes.")
        return

    target_class_id = classes.index(target_class_name)

    # Perform object detection
    detections = detect_objects(image, net, output_layers, conf_threshold=0.5)

    # Save bounding boxes as separate images
    save_bounding_boxes(image, detections, classes, target_class_id, output_dir, prefix)

def identify_and_separate_objects(left_image_path, right_image_path, yolo_model, target_class_name="pottedplant"):
    """
    Identifies objects in stereo images using YOLO and separates them into individual image pairs.
    """
    # Process left image and save crops
    process_and_save(
        left_image_path,
        yolo_model,
        target_class_name,
        output_dir="output_left",
        prefix="left"
    )

    # Process right image and save crops
    process_and_save(
        right_image_path,
        yolo_model,
        target_class_name,
        output_dir="output_right",
        prefix="right"
    )

if __name__ == "__main__":
    # Paths to YOLO configuration, weights, and class names
    cfg_path = "yolov4.cfg"
    weights_path = "yolov4.weights"
    names_path = "coco.names"

    # Load YOLO model
    yolo_model = load_yolo_model(cfg_path, weights_path, names_path)

    # Call the function with stereo image paths and target class name
    identify_and_separate_objects("IMG_7998.jpg", "IMG_7999.jpg", yolo_model, target_class_name="pottedplant")


Saved: output_left\left_object_0.jpg
Saved: output_left\left_object_1.jpg
Saved: output_right\right_object_0.jpg
Saved: output_right\right_object_1.jpg
